In [1]:
import requests
import pandas as pd

coins = []

for page in [1, 2]:  # 250 moedas por página × 2 = 500 moedas
    url = (
        f"https://api.coingecko.com/api/v3/coins/markets"
        f"?vs_currency=usd&order=market_cap_desc&per_page=250&page={page}"
    )
    response = requests.get(url)
    if response.status_code == 200:
        coins.extend(response.json())
    else:
        print(f"Erro ao buscar página {page}: {response.status_code}")

# Criar DataFrame
df = pd.DataFrame(coins)

# Mostrar as colunas principais
df_filtered = df[['market_cap_rank', 'id', 'name', 'symbol', 'current_price']]
df.head(500)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,105131.000000,2089440518744,1,2.089441e+12,2.695141e+10,106353.000000,...,1.987433e+07,2.100000e+07,111814.000000,-5.97738,2025-05-22T18:41:28.492Z,67.810000,1.549393e+05,2013-07-06T00:00:00.000Z,None,2025-06-04T18:00:57.366Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2637.190000,318373450770,2,3.183735e+11,1.603150e+10,2667.170000,...,1.207229e+08,NaN,4878.260000,-45.93990,2021-11-10T14:24:19.604Z,0.432979,6.089811e+05,2015-10-20T00:00:00.000Z,"{'times': 32.55236121746746, 'currency': 'btc'...",2025-06-04T18:00:56.777Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.000000,153770797479,3,1.537708e+11,4.510717e+10,1.001000,...,1.537207e+11,NaN,1.320000,-24.39510,2018-07-24T00:00:00.000Z,0.572521,7.472283e+01,2015-03-02T00:00:00.000Z,None,2025-06-04T18:00:52.832Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.230000,131172540480,4,2.232044e+11,1.839999e+09,2.280000,...,9.998608e+10,1.000000e+11,3.400000,-34.30769,2018-01-07T00:00:00.000Z,0.002686,8.301052e+04,2014-05-22T00:00:00.000Z,None,2025-06-04T18:00:57.173Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,667.560000,97385106786,5,9.738511e+10,5.867515e+08,671.690000,...,1.458876e+08,2.000000e+08,788.840000,-15.37544,2024-12-04T10:35:25.220Z,0.039818,1.676429e+06,2017-10-19T00:00:00.000Z,None,2025-06-04T18:00:57.585Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,aevo-exchange,aevo,Aevo,https://coin-images.coingecko.com/coins/images...,0.102724,92927180,496,1.027388e+08,1.789174e+07,0.106441,...,1.000000e+09,1.000000e+09,3.760000,-97.26270,2024-03-28T00:05:46.673Z,0.076023,3.525253e+01,2025-04-09T01:32:01.776Z,None,2025-06-04T18:01:09.212Z
496,neiro-on-eth,neiro,Neiro on ETH,https://coin-images.coingecko.com/coins/images...,0.092555,92535784,497,9.253578e+07,9.123812e+06,0.095287,...,1.000000e+09,1.000000e+09,0.293273,-68.31682,2024-08-07T16:53:31.670Z,0.014213,5.537382e+02,2025-02-03T02:55:51.693Z,None,2025-06-04T18:01:10.345Z
497,audius,audio,Audius,https://coin-images.coingecko.com/coins/images...,0.069531,92407820,498,9.240782e+07,5.701536e+06,0.070869,...,1.329378e+09,1.329378e+09,4.950000,-98.60487,2021-03-27T15:45:35.137Z,0.043737,5.804813e+01,2020-10-23T14:56:23.999Z,None,2025-06-04T18:01:13.644Z
498,songbird,sgb,Songbird,https://coin-images.coingecko.com/coins/images...,0.005832,91842559,499,1.090030e+08,4.805600e+05,0.006058,...,1.868915e+10,NaN,0.713340,-99.18267,2021-09-30T06:32:35.621Z,0.003718,5.682404e+01,2023-10-13T10:55:52.860Z,None,2025-06-04T18:01:12.841Z


In [2]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# Intervalo de datas: últimos 365 dias
end_date = datetime.now()
start_date = end_date - timedelta(days=365)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Seu DataFrame original com coluna 'id'
# df = pd.read_csv("seu_arquivo.csv")  # se necessário
crypto_ids = df.head(100)['id'].tolist()

# Pegar lista de coins da CoinGecko
cg_list_url = "https://api.coingecko.com/api/v3/coins/list"
cg_response = requests.get(cg_list_url)

if cg_response.status_code != 200:
    raise Exception("Erro ao buscar lista de moedas da CoinGecko")

cg_ids = pd.DataFrame(cg_response.json())  # contém 'id', 'symbol', 'name'
cg_set = set(cg_ids['id'])

historical_data = {}

for crypto_id in crypto_ids:
    if crypto_id in cg_set:
        url = (
            f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
            f"?vs_currency=usd&from={start_timestamp}&to={end_timestamp}"
        )
        try:
            resp = requests.get(url)
            if resp.status_code == 200:
                prices = resp.json()['prices']
                # Salva a série de preços no dict com a data como chave
                daily_prices = {datetime.utcfromtimestamp(p[0]/1000).strftime('%Y-%m-%d'): p[1] for p in prices}
                historical_data[crypto_id] = daily_prices
                print(f"Dados históricos para {crypto_id} obtidos com sucesso com {len(daily_prices)} dias com dados.")
            else:
                historical_data[crypto_id] = {}
        except Exception as e:
            historical_data[crypto_id] = {}
        time.sleep(20)  # controle de taxa (20 segundos)
    else:
        historical_data[crypto_id] = {}  # sem correspondência no CoinGecko

# Exportar para JSON
with open("historical_data.json", "w") as f:
    import json
    json.dump(historical_data, f, indent=2)

# Criar DataFrame
df_hist = pd.DataFrame(historical_data).T  # linhas: ids, colunas: datas
df_hist.index.name = "crypto_id"

# Exibir as primeiras linhas da tabela final
print(df_hist.head())

C:\Users\dartf\AppData\Local\Temp\ipykernel_9272\1042018528.py:39: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  daily_prices = {datetime.utcfromtimestamp(p[0]/1000).strftime('%Y-%m-%d'): p[1] for p in prices}


Dados históricos para bitcoin obtidos com sucesso com 365 dias com dados.
Dados históricos para ethereum obtidos com sucesso com 365 dias com dados.
Dados históricos para tether obtidos com sucesso com 365 dias com dados.
Dados históricos para ripple obtidos com sucesso com 365 dias com dados.
Dados históricos para binancecoin obtidos com sucesso com 365 dias com dados.
Dados históricos para solana obtidos com sucesso com 365 dias com dados.
Dados históricos para usd-coin obtidos com sucesso com 365 dias com dados.
Dados históricos para dogecoin obtidos com sucesso com 365 dias com dados.
Dados históricos para tron obtidos com sucesso com 365 dias com dados.
Dados históricos para cardano obtidos com sucesso com 365 dias com dados.
Dados históricos para staked-ether obtidos com sucesso com 365 dias com dados.
Dados históricos para wrapped-bitcoin obtidos com sucesso com 365 dias com dados.
Dados históricos para hyperliquid obtidos com sucesso com 188 dias com dados.
Dados históricos par

In [3]:
df_hist = df_hist.round(2)

In [4]:
df_hist.to_csv("historical_data.csv", index=True)

In [5]:
import sqlite3

# Conectar ou criar o banco
conn = sqlite3.connect("crypt.db")
cursor = conn.cursor()

# Criar tabela de criptomoedas
cursor.execute("""
CREATE TABLE IF NOT EXISTS cryptocurrencies (
    id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    symbol TEXT NOT NULL,
    market_cap_rank INTEGER,
    current_price REAL
)
""")

# Criar tabela de histórico de preços
cursor.execute("""
CREATE TABLE IF NOT EXISTS price_history (
    crypto_id TEXT NOT NULL,
    price_date TEXT NOT NULL,
    price_usd REAL NOT NULL,
    PRIMARY KEY (crypto_id, price_date),
    FOREIGN KEY (crypto_id) REFERENCES cryptocurrencies(id)
)
""")

conn.commit()
conn.close()

In [6]:
import sqlite3

# Conecta ao banco
conn = sqlite3.connect("crypt.db")
cursor = conn.cursor()

# Insere cada linha do df_filtered na tabela
for _, row in df_filtered.iterrows():
    cursor.execute("""
        INSERT OR REPLACE INTO cryptocurrencies (id, name, symbol, market_cap_rank, current_price)
        VALUES (?, ?, ?, ?, ?)
    """, (
        row['id'],
        row['name'],
        row['symbol'],
        int(row['market_cap_rank']) if not pd.isna(row['market_cap_rank']) else None,
        float(row['current_price']) if not pd.isna(row['current_price']) else None
    ))

conn.commit()
conn.close()


In [7]:
import sqlite3

# Conecta ao banco
conn = sqlite3.connect("crypt.db")
cursor = conn.cursor()

# Loop sobre cada moeda e seus preços diários
for crypto_id, prices in historical_data.items():
    for date, price in prices.items():
        cursor.execute("""
            INSERT OR REPLACE INTO price_history (crypto_id, price_date, price_usd)
            VALUES (?, ?, ?)
        """, (crypto_id, date, float(price)))

conn.commit()
conn.close()


In [8]:
conn = sqlite3.connect("crypt.db")
df = pd.read_sql("SELECT * FROM price_history LIMIT 10", conn)
df['price_usd'] = df['price_usd'].round(2)
print(df)
conn.close()

  crypto_id  price_date  price_usd
0   bitcoin  2024-06-05   70600.01
1   bitcoin  2024-06-06   71184.60
2   bitcoin  2024-06-07   70759.59
3   bitcoin  2024-06-08   69325.36
4   bitcoin  2024-06-09   69315.10
5   bitcoin  2024-06-10   69654.16
6   bitcoin  2024-06-11   69493.18
7   bitcoin  2024-06-12   67329.15
8   bitcoin  2024-06-13   68224.52
9   bitcoin  2024-06-14   66700.32


In [9]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("crypt.db")

# Exportar tabela cryptocurrencies
df_cryptos = pd.read_sql("SELECT * FROM cryptocurrencies", conn)
df_cryptos.to_csv("cryptocurrencies.csv", index=False)

# Exportar tabela price_history
df_prices = pd.read_sql("SELECT * FROM price_history", conn)
df_prices.to_csv("price_hist.csv", index=False, float_format="%.2f")

conn.close()